In [3]:
!pip install novita-client --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import requests
import io
from PIL import Image

from novita_client import NovitaClient, FACE_TRAINING_DEFAULT_COMPONENTS, Txt2ImgRequest, ADEtailer, Samplers

In [4]:
novita_client = NovitaClient(api_key="650YpEeEBF2H88Z88idG6ZWvWiU2eVG6") # get api key from https://novita.ai

In [5]:
def load_image(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content))
    return img


def make_image_grid(images, rows: int, cols: int, resize: int = None):
    """
    Prepares a single grid of images. Useful for visualization purposes.
    """
    assert len(images) == rows * cols

    if resize is not None:
        images = [img.resize((resize, resize)) for img in images]

    w, h = images[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))

    for i, img in enumerate(images):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid



In [7]:
man_images_urls = [
  "https://github.com/novitalabs/python-sdk/assets/16937838/d5bd9408-f00f-40af-9ab9-a08d9e3bab4c",
  "https://github.com/novitalabs/python-sdk/assets/16937838/de917152-0da9-45e7-b739-39fe0846533c",
  "https://github.com/novitalabs/python-sdk/assets/16937838/d54450f5-0b2b-4660-8c28-a46d07d63116",
  "https://github.com/novitalabs/python-sdk/assets/16937838/d9be4198-ee4c-438e-829b-fd64d8f7c0fc",
  "https://github.com/novitalabs/python-sdk/assets/16937838/f957cdff-c7cb-43a2-974b-b34f8b75b991",
  "https://github.com/novitalabs/python-sdk/assets/16937838/5605e479-0834-4394-9e86-96dd95d5345e",
  "https://github.com/novitalabs/python-sdk/assets/16937838/617c7767-3e9a-4f02-bfc0-e5abd0e95847",
  "https://github.com/novitalabs/python-sdk/assets/16937838/a9e388f9-90b5-42fc-aab0-fdbe72fc9072",
  "https://github.com/novitalabs/python-sdk/assets/16937838/9a1a2182-9bfe-49ae-bb9f-2fea5dbbc33a",
  "https://github.com/novitalabs/python-sdk/assets/16937838/adef6547-82d7-4ee2-9f85-129a927cc5c7",
  "https://github.com/novitalabs/python-sdk/assets/16937838/08587977-1ca6-474d-90c0-5feeb955c8f1",
  "https://github.com/novitalabs/python-sdk/assets/16937838/84e34e7d-4603-4191-9943-2f02ae5d8f93"
]


man_images = []

for image_url in man_images_urls:
    man_images.append(load_image(image_url))

In [8]:
make_image_grid(man_images, 3, 4, None)

In [8]:
train_task_id = novita_client.create_training_subject(name="whiteman101", 
                                      base_model="epicrealism_naturalSin_121250", 
                                      images=man_images, 
                                      instance_prompt="a closeup photo of ohwx person", 
                                      class_prompt="person", 
                                      max_train_steps=2000, 
                                      components=FACE_TRAINING_DEFAULT_COMPONENTS,
                                      learning_rate=3e-4,
                                      seed=None,
                                      lr_scheduler='cosine_with_restarts',
                                      with_prior_preservation=True,
                                      prior_loss_weight=1.0,
                                      width=512,
                                      height=512,
                                      lora_r=32,
                                      lora_alpha=32,
                                      lora_text_encoder_r=32,
                                      lora_text_encoder_alpha=32)

NameError: name 'man_images' is not defined

In [7]:
import time 
train_model_name = None
while True:
    train_task_status = novita_client.query_training_subject_status(train_task_id)
    if train_task_status.task_status == "TRAINING" or train_task_status.task_status == "QUEUING":
        print("Training in progress...")
        time.sleep(30)
    elif train_task_status.task_status == "SUCCESS":
        train_model_name = train_task_status.models[0].model_name
        print(f"Training completed successfully! Model id: {train_task_status.models[0].model_name}")
        break
    elif train_task_status.task_status == "FAILED":
        print("Training failed!")
        break
    else:
        print(f"Unknown training status: {train_task_status.task_status}")
        break

train_model_name = train_model_name.replace(".safetensors", "")

NameError: name 'train_task_id' is not defined

In [6]:
first_stage_lora_scale = 0.5

second_stage_lora_scale = 1.0
second_stage_strength = 0.55

txt2img_res = novita_client.sync_txt2img(
  Txt2ImgRequest(
    # prompt=f"a photo of ohwx man, handsome, <lora:{train_model_name}:{first_stage_lora_scale}>",
    prompt=f"a photo of ohwx man,  <lora:{train_model_name}:{first_stage_lora_scale}>, (extremely intricate:1.3), (realistic), portrait of a person, the most handsome in the world, (medieval armor), metal reflections, upper body, outdoors, intense sunlight, far away castle, professional photograph of a stunning person detailed, sharp focus, dramatic, award winning, cinematic lighting, octane render unreal engine, volumetrics dtx, (film grain, blurry background, blurry foreground, bokeh, depth of field, sunset, motion blur:1.3), chainmail",
    sampler_name=Samplers.DPMPP_M_KARRAS,
    model_name="epicrealism_pureEvolutionV5_97793.safetensors",
    height=768,
    width=512,
    steps=20,
    batch_size=4,
    # hr_upscaler='Latent',
    # enable_hr=True,
    adetailer=ADEtailer(
      prompt=f"a closeup photo of ohwx man, masterpiece, <lora:{train_model_name}:{second_stage_lora_scale}>",
      steps=20,
      strength=second_stage_strength)
    )
)

NameError: name 'train_model_name' is not defined

In [ ]:
res_images = []

for img_bytes in txt2img_res.data.imgs_bytes:
    res_images.append(Image.open(io.BytesIO(img_bytes)))

make_image_grid(res_images, 1, 4)


NameError: name 'txt2img_res' is not defined